In [2]:
import numpy as np
import pandas as pd

import sys
import os
import random

from PIL import Image, ImageFont, ImageDraw
from PIL.ImageChops import add, subtract, multiply, difference, screen

import cv2

import matplotlib.pyplot as plt

# OpenCV

## Image Overlays Using Bitwise Operations in OpenCV

In [3]:
home = os.environ['HOME']
path = os.path.join(home, 'git', 'deep_food', 'gleb_google_images')
path

'/Users/iskriyanavasileva/git/deep_food/gleb_google_images'

In [3]:
def load_bg_random(path, bg_folder='background'):
    """
    chooses a random background image
    """    
    bg_folder = os.path.join(path, bg_folder)
    bg_files = os.listdir(bg_folder)
    bg_index = random.randrange(0, len(bg_files))
    
    bg = os.path.join(path, bg_folder, bg_files[bg_index])
    
    background = cv2.imread(bg)
    rows_b, cols_b, channels_b = background.shape
    
    return background, rows_b, cols_b, channels_b 

In [4]:
def load_img_mask(path):
    """
    creates folder for combined images
    loads the images by category and labels them
    creates a mask of the ingredient
    creates a new combined image
    """    
    for root, dirs, files in os.walk(path):
        #dirs[:] = [d for d in dirs if not d[0] == '.']
        #for dir in dirs:
            path_comb = os.path.join(path, str(dir) + '_comb')
            #dir_comb = os.makedirs(path_comb, exist_ok=True)
            files = [f for f in files if not f[0] == '.']
            for file in files:
                file_path = os.path.join(root, file)
                ingredient = cv2.imread(file_path)
                
                #load random background
                background, rows_b, cols_b, channels_b = load_bg_random(path)
                
                # resize ingredient image to make sure it is always smaller than the background
                # keep the aspect ratio by resizing                
                aspect_ratio = float(ingredient.shape[1])/float(ingredient.shape[0])
                rows_i = int(rows_b/2)
                cols_i = int(rows_i/aspect_ratio)
                dim_res = (rows_i, cols_i)
                ingredient = cv2.resize(ingredient, dim_res)
                
                # Create a Region of Images (=ROI) in a randomised matter 
                rows, cols, channels = ingredient.shape

                # TODO position on picture needs to be randomised
                #start_rows = np.random.randint(0, size=rows)
                #start_cols = np.random.randint(0, size=cols)
                #roi = background[start_rows:rows, start_cols:cols] 
                roi = background[0:rows, 0:cols]

                # Now create a mask of the ingredient and create its inverse mask also
                img2gray = cv2.cvtColor(ingredient, cv2.COLOR_BGR2GRAY)

                # everything higher than the threshold (i.e. lighter), will become black
                # the rest (i.e. the darker pixels filled by the object), will become white
                # with THRESH_OTSU the threshold is adjustable according to the image
                # similar to the idea to take the mean value between black and white as a threshold
                # the GaussianBluring smooths the image to make the black & white more precise
                blur = cv2.GaussianBlur(img2gray, (5,5), 0)
                ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                mask_inv = cv2.bitwise_not(mask)


                # Black-out the area of the ingredient in ROI in the background image
                background_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)

                # Take only region of the ingredient from the ingredient image.
                ingredient_fg = cv2.bitwise_and(ingredient, ingredient, mask = mask)

                # Put the ingredient in ROI and modify the main image
                dst = cv2.add(background_bg, ingredient_fg)

                #TODO positions need to be randomised (see above)
                #background[start_rows:rows, start_cols:cols] = dst
                background[0:rows, 0:cols] = dst
                
                img_comb = background 
                             
                                         
                #filename = os.path.join(path_comb, f'{file}_comb.jpg')
                file = os.path.splitext(file)[0]
                filename = os.path.join(root, f'{file}_comb.jpg')
                cv2.imwrite(filename, img_comb)
                print(filename)

In [5]:
load_img_mask(path)

In [6]:
# Load two images - background and ingredient 



ingredient = cv2.imread('../test_images/banana.jpg')

# resize ingredient image to make sure it is always smaller than the background
# keep the aspect ratio by resizing https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
aspect_ratio = float(ingredient.shape[1])/float(ingredient.shape[0])
rows_i = int(rows_b/2)
cols_i = int(rows_i/aspect_ratio)
dim_res = (rows_i, cols_i)
ingredient = cv2.resize(ingredient, dim_res)


# Create a Region of Images (=ROI) in a randomised matter 
# roi = Region Of Images https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_core/py_basic_ops/py_basic_ops.html
rows, cols, channels = ingredient.shape

# TODO position on picture needs to be randomised
#start_rows = np.random.randint(0, size=rows)
#start_cols = np.random.randint(0, size=cols)
#roi = background[start_rows:rows, start_cols:cols] 
roi = background[0:rows, 0:cols]



# Now create a mask of the ingredient and create its inverse mask also
img2gray = cv2.cvtColor(ingredient, cv2.COLOR_BGR2GRAY)

# everything higher than the threshold (i.e. lighter), will become black
# the rest (i.e. the darker pixels filled by the object), will become white
# with THRESH_OTSU the threshold is adjustable according to the image
# similar to the idea to take the mean value between black and white as a threshold
# the GaussianBluring smooths the image to make the black & white more precise
blur = cv2.GaussianBlur(img2gray, (5,5), 0)
ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
mask_inv = cv2.bitwise_not(mask)


# Black-out the area of the ingredient in ROI in the background image
background_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)

# Take only region of the ingredient from the ingredient image.
ingredient_fg = cv2.bitwise_and(ingredient, ingredient, mask = mask)

# Put the ingredient in ROI and modify the main image
dst = cv2.add(background_bg, ingredient_fg)

#TODO positions need to be randomised (see above)
#background[start_rows:rows, start_cols:cols] = dst
background[0:rows, 0:cols] = dst

cv2.imshow('res', background)
#cv2.imshow('img2gray',img2gray)
#cv2.imshow('mask', mask)
#cv2.imshow('mask_inv', mask_inv)
#cv2.imshow('ret', ret)
#cv2.imshow('background_bg', background_bg)
#cv2.imshow('ingredient_fg', ingredient_fg)
#cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

NameError: name 'rows_b' is not defined

In [18]:
for root, dirs, files in os.walk('./git/deep-food/gleb_images_google'):
    print(root)
        #for file in files:
        #    print(file)

In [7]:
dirl

NameError: name 'dirl' is not defined

In [8]:
dirs

[]

In [11]:
test = [os.walk(path)]

[<generator object walk at 0x7ffb194acbf8>]

In [14]:
for root, dirs, files in os.walk('./git/deep-food/gleb_images_google'):
    print(root, dirs, files)